# Assignement for course ML2

Students: Tristan STAMPFLER and Thomas CARTA

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("data/CreditTraining.csv")

## 1) Exploration of the dataset

Preliminary exploration of the dataset is necessary to determine which field we can use.

In [3]:
df.head()

,Id_Customer,Y,Customer_Type,BirthDate,Customer_Open_Date,P_Client,Educational_Level,Marital_Status,Number_Of_Dependant,Years_At_Residence,Net_Annual_Income,Years_At_Business,Prod_Sub_Category,Prod_Decision_Date,Source,Type_Of_Residence,Nb_Of_Products,Prod_Closed_Date,Prod_Category
0,7440,0,Non Existing Client,07/08/1977,13/02/2012,NP_Client,University,Married,3.0,1,36,1.0,C,14/02/2012,Sales,Owned,1,NaN,B
1,573,0,Existing Client,13/06/1974,04/02/2009,P_Client,University,Married,0.0,12,18,2.0,C,30/06/2011,Sales,Parents,1,NaN,G
2,9194,0,Non Existing Client,07/11/1973,03/04/2012,NP_Client,University,Married,2.0,10,36,1.0,C,04/04/2012,Sales,Owned,1,NaN,B
3,3016,1,Existing Client,08/07/1982,25/08/2011,NP_Client,University,Married,3.0,3,36,1.0,C,07/09/2011,Sales,New rent,1,31/12/2012,L
4,6524,0,Non Existing Client,18/08/1953,10/01/2012,NP_Client,University,Married,2.0,1,36,1.0,C,11/01/2012,Sales,Owned,1,NaN,D


In [4]:
df.describe()

,Id_Customer,Y,Number_Of_Dependant,Years_At_Residence,Years_At_Business,Nb_Of_Products
count,5380.000000,5380.000000,5378.000000,5380.000000,5378.000000,5380.000000
mean,4784.535688,0.073048,1.058572,12.626022,4.266270,1.089033
std,2781.436262,0.260240,1.339001,9.972164,7.225926,0.297587
min,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,2368.500000,0.000000,0.000000,4.000000,1.000000,1.000000
50%,4762.500000,0.000000,0.000000,10.000000,1.000000,1.000000
75%,7180.250000,0.000000,2.000000,18.000000,4.000000,1.000000
max,9605.000000,1.000000,20.000000,70.000000,98.000000,3.000000


### Checking for NaNs

In [5]:
df.isna().sum()

Id_Customer               0
Y                         0
Customer_Type             0
BirthDate                 0
Customer_Open_Date        0
P_Client                  0
Educational_Level         0
Marital_Status            0
Number_Of_Dependant       2
Years_At_Residence        0
Net_Annual_Income         2
Years_At_Business         2
Prod_Sub_Category         0
Prod_Decision_Date        0
Source                    0
Type_Of_Residence         0
Nb_Of_Products            0
Prod_Closed_Date       4206
Prod_Category             0
dtype: int64

We can see have a lot of missing values for the field Prod_Closed_Date, and some for Number_Of_Dependant, Net_Annual_Income and Years_At_Business

### Checking if Id_Customer is an unique key

In [6]:
print(df[['Id_Customer']].nunique() == len(df))

Id_Customer    True
dtype: bool


Yes, Id_Customer is a unique key for this dataset

### Practising on marital

In [7]:
df_marital = df[['Id_Customer','Y','Marital_Status']]
df_marital['Marital_Status'].value_counts()

Married      4206
Single       1046
Widowed        64
Divorced       63
Separated       1
Name: Marital_Status, dtype: int64

In [8]:
marital_grp = df_marital.groupby('Marital_Status')
marital_grp['Y'].value_counts(normalize=True)

Marital_Status  Y
Divorced        0    0.920635
                1    0.079365
Married         0    0.930813
                1    0.069187
Separated       0    1.000000
Single          0    0.911090
                1    0.088910
Widowed         0    0.937500
                1    0.062500
Name: Y, dtype: float64

In [9]:
marital_grp.agg({'Y':'mean',
                 'Id_Customer':'count'})

,Y,Id_Customer
Marital_Status,,
Divorced,0.079365,63
Married,0.069187,4206
Separated,0.000000,1
Single,0.088910,1046
Widowed,0.062500,64


## 2) Data cleaning

We need to handle NaNs and select the columns we will use for our approaches

In [10]:
df.isna().sum()

Id_Customer               0
Y                         0
Customer_Type             0
BirthDate                 0
Customer_Open_Date        0
P_Client                  0
Educational_Level         0
Marital_Status            0
Number_Of_Dependant       2
Years_At_Residence        0
Net_Annual_Income         2
Years_At_Business         2
Prod_Sub_Category         0
Prod_Decision_Date        0
Source                    0
Type_Of_Residence         0
Nb_Of_Products            0
Prod_Closed_Date       4206
Prod_Category             0
dtype: int64

### Removing NaNs

We will remove NaNs in 2 ways. For Prod_Closed_Date there are too much values missing so we will just get rid of this field. For Number_Of_Dependant, Net_Annual_Income and Years_At_Business we just have a few values missing. Assuming the clients are iid, we can remove the rows with missing values.

In [11]:
df = df.drop("Prod_Closed_Date", axis=1)
df = df.dropna()

In [12]:
df = df.reset_index(drop=True)
df.isna().sum()
df.head()

,Id_Customer,Y,Customer_Type,BirthDate,Customer_Open_Date,P_Client,Educational_Level,Marital_Status,Number_Of_Dependant,Years_At_Residence,Net_Annual_Income,Years_At_Business,Prod_Sub_Category,Prod_Decision_Date,Source,Type_Of_Residence,Nb_Of_Products,Prod_Category
0,7440,0,Non Existing Client,07/08/1977,13/02/2012,NP_Client,University,Married,3.0,1,36,1.0,C,14/02/2012,Sales,Owned,1,B
1,573,0,Existing Client,13/06/1974,04/02/2009,P_Client,University,Married,0.0,12,18,2.0,C,30/06/2011,Sales,Parents,1,G
2,9194,0,Non Existing Client,07/11/1973,03/04/2012,NP_Client,University,Married,2.0,10,36,1.0,C,04/04/2012,Sales,Owned,1,B
3,3016,1,Existing Client,08/07/1982,25/08/2011,NP_Client,University,Married,3.0,3,36,1.0,C,07/09/2011,Sales,New rent,1,L
4,6524,0,Non Existing Client,18/08/1953,10/01/2012,NP_Client,University,Married,2.0,1,36,1.0,C,11/01/2012,Sales,Owned,1,D


## 3) Feature engineering

We create some features from the fields we have in our dataset.
<br>
We compute here for instance the age of the client. We then get rid of obsolete fields.

In [13]:
df['Age'] = (pd.Timestamp('2020-01-01 00:00:00.000000') - pd.to_datetime(df['BirthDate'])).dt.days / 365.25
df = df.drop('BirthDate', axis=1)

## 4) Feature selection

In [14]:
df.head()

,Id_Customer,Y,Customer_Type,Customer_Open_Date,P_Client,Educational_Level,Marital_Status,Number_Of_Dependant,Years_At_Residence,Net_Annual_Income,Years_At_Business,Prod_Sub_Category,Prod_Decision_Date,Source,Type_Of_Residence,Nb_Of_Products,Prod_Category,Age
0,7440,0,Non Existing Client,13/02/2012,NP_Client,University,Married,3.0,1,36,1.0,C,14/02/2012,Sales,Owned,1,B,42.483231
1,573,0,Existing Client,04/02/2009,P_Client,University,Married,0.0,12,18,2.0,C,30/06/2011,Sales,Parents,1,G,45.552361
2,9194,0,Non Existing Client,03/04/2012,NP_Client,University,Married,2.0,10,36,1.0,C,04/04/2012,Sales,Owned,1,B,46.475017
3,3016,1,Existing Client,25/08/2011,NP_Client,University,Married,3.0,3,36,1.0,C,07/09/2011,Sales,New rent,1,L,37.401780
4,6524,0,Non Existing Client,10/01/2012,NP_Client,University,Married,2.0,1,36,1.0,C,11/01/2012,Sales,Owned,1,D,66.370979


In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [25]:
X = df.loc[:,['Age']]
Y = df.loc[:,'Y']
clf = LogisticRegression()
clf.fit(X, Y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [31]:
Y_predict = clf.predict(X)
accuracy_score(Y, Y_predict)

0.9270561965016747